In [1]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=9071e08fb624dda8fa9cf68c88076089d1ce6176ed2a158d7fa95b8c4736bcce
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [3]:
import os
import requests
import json
import openai
import openpyxl
import pandas as pd
import spacy
from spacy.lang.en import English

openai.api_key = "YOUR API KEY"
openai.api_base =  "https://YOUR_RESOURCE_NAME.openai.azure.com/" # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2022-12-01' # this may change in the future
deployment_name='YOUR DEPLOYMENT NAME' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'how are u \n'
response = openai.Completion.create(engine=deployment_name, prompt=start_phrase, max_tokens=10)
text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(start_phrase+text) 

Sending a test completion job
how are u 
I'm doing great, thanks for asking!


In [4]:
# read csv
df = pd.read_csv(r'/content/input.csv')
print(df)

In [5]:
df.drop_duplicates

<bound method DataFrame.drop_duplicates of                                           Review Title  \
0         Another Midrange killer Smartphone by Xiaomi   
1                                vry small size mobile   
2         Full display not working in all application.   
3                                      Value for Money   
4                              Not worth for the money   
..                                                 ...   
275  Cemera quality,face unlock most important in t...   
276                                   Mi is best phone   
277                                     Its a OK Phone   
278                                              Redmi   
279                           Not worth for the price.   

                Customer name              Rating                  Date  \
0           Rishikumar Thakur  4.0 out of 5 stars     on 1 October 2018   
1                     Raza ji  3.0 out of 5 stars  on 15 September 2018   
2               Vaibhav Patel  3.0 

In [6]:
#Sentiment_for_each_comment
def CommentReview(comments):
    completions=openai.Completion.create(
        engine=deployment_name,
        prompt=(f"what type of comment - {comments} it is negative, positive or neutral?"),
        max_tokens=20
    )
    message=completions['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return message

cols = ['Category', 'Comments', 'ReviewType']
df2 = pd.DataFrame(columns=cols)
Sentiment_for_each_comment = pd.DataFrame(columns=cols)

for ind in df.index:
    cat = df['Category'][ind]
    com = df['Comments'][ind]
    Sentiment_for_each_comment = Sentiment_for_each_comment.append({'Category': df['Category'][ind], 'Comments':df['Comments'][ind], 'ReviewType': CommentReview(df['Comments'][ind])},ignore_index=True)


In [7]:
df_grouped = df.groupby("Category")["Comments"].apply(" ".join).reset_index()
df_grouped = df_grouped.rename(columns={"Comments": "Aggregated_Comments"})
df_grouped

,Category,Aggregated_Comments
0,Battery,"Thanks alot Amazon,ecom & seller.perfect produ..."
1,Camera,Front camera is poor rest things are good Fron...
2,Delivery,Nice product. Delivery guy also. Nice product....
3,Display,Another Midrange killer Smartphone by Xiaomi\n...
4,Others,All ok but vry small size mobile Quite good wo...


In [8]:
#category wise summariszation of comments

nlp = spacy.load("en_core_web_sm")

def text_to_chunks(text):
  chunks = [[]]
  chunk_total_words = 0

  sentences = nlp(text)

  for sentence in sentences.sents:
    chunk_total_words += len(sentence.text.split(" "))

    if chunk_total_words > 1000:
      chunks.append([])
      chunk_total_words = len(sentence.text.split(" "))

    chunks[len(chunks)-1].append(sentence.text)
  
  return chunks

def Comment_category(Comments):
    completions=openai.Completion.create(
        engine=deployment_name,
        prompt=(f"Give Summariaztion for the following comments:{Comments}"),
        max_tokens=200
    )
    message=completions['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
    return message

print("Category wise Summarization of comments:")

cols = ['Category','Summarized Comment']
summariszation_of_comments = pd.DataFrame(columns=cols)

for ind in df_grouped.index:
    cat = df_grouped['Category'][ind]
    com = df_grouped['Aggregated_Comments'][ind]
    chunks = text_to_chunks(com)
    for chunk in chunks:
      chunk_summaries = []
      chunk_summary = Comment_category(" ".join(chunk))
      chunk_summaries.append(chunk_summary)
    summary = " ".join(chunk_summaries)
    print(df_grouped['Category'][ind], ":"+"\n"+Comment_category(summary), end="\n")
    summariszation_of_comments = summariszation_of_comments.append({'Category': df['Category'][ind], 'Summarized Comment': Comment_category(summary)},ignore_index=True)

Category wise Summarization of comments:
Battery :
The Mi 6 Pro is a satisfactory phone considering its price and quality; however, it has some issues such as lagging, hanging, and charging. The Redmi Note 4 is performing better than the newer models, but the Mi 6 Pro is still considered an okay phone for its price point. Nevertheless, customers are disappointed with the lack of additional features, such as no front flash due to the smaller USB cables.
Camera :
This phone has a notch design and poor front camera quality, an outdated processor that affects use of multiple apps, a suboptimal build quality susceptible to damage, an average rear camera quality, distorted speakerphone and earphone, and a decent screen that provides a poorer video quality when streaming. At the 11K price tag, this phone is not worth it and other better options are in the same range.
Delivery :
This customer was overall pleased with the product they received from Amazon, finding it to be powerful and with qui

In [9]:
file_name = 'output.xlsx'
# Load the workbook
writer = pd.ExcelWriter('output.xlsx', engine='openpyxl') 
workbook = openpyxl.Workbook()
writer.book = workbook

# Write the DataFrame to a new sheet
Sentiment_for_each_comment.to_excel(writer, index=False, sheet_name='Sentiment for each comment')
summariszation_of_comments.to_excel(writer, index=False, sheet_name='summariszation of comments')

# Save the workbook
writer.save()
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
